# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 3 2022 8:30AM,243310,10,MSP212748,"Methapharm, Inc.",Released
1,Aug 3 2022 8:30AM,243310,10,MSP212749,"Methapharm, Inc.",Released
2,Aug 3 2022 8:30AM,243310,10,MSP212750,"Methapharm, Inc.",Released
3,Aug 3 2022 8:30AM,243310,10,MSP212736,"Methapharm, Inc.",Released
4,Aug 3 2022 8:30AM,243310,10,MSP212752,"Methapharm, Inc.",Released
5,Aug 3 2022 8:30AM,243310,10,MSP212753,"Methapharm, Inc.",Released
6,Aug 3 2022 8:30AM,243310,10,MSP212754,"Methapharm, Inc.",Released
7,Aug 3 2022 8:30AM,243310,10,MSP212755,"Methapharm, Inc.",Released
8,Aug 3 2022 8:30AM,243310,10,MSP212756,"Methapharm, Inc.",Released
9,Aug 3 2022 8:30AM,243310,10,MSP212757,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,243315,Released,1
31,243316,Released,1
32,243317,Released,1
33,243318,Released,1
34,243319,Released,20


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243315,NaN,NaN,1.0
243316,NaN,NaN,1.0
243317,NaN,NaN,1.0
243318,NaN,NaN,1.0
243319,NaN,NaN,20.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243174,14.0,21.0,2.0
243210,5.0,6.0,2.0
243218,0.0,1.0,0.0
243241,0.0,1.0,0.0
243259,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243174,14,21,2
243210,5,6,2
243218,0,1,0
243241,0,1,0
243259,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243174,14,21,2
1,243210,5,6,2
2,243218,0,1,0
3,243241,0,1,0
4,243259,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243174,14,21,2
1,243210,5,6,2
2,243218,,1,
3,243241,,1,
4,243259,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 3 2022 8:30AM,243310,10,"Methapharm, Inc."
19,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation
27,Aug 3 2022 8:30AM,243316,10,"Nextsource Biotechnology, LLC"
29,Aug 3 2022 8:30AM,243312,10,SugarBear
30,Aug 2 2022 4:04PM,243319,19,"AdvaGen Pharma, Ltd"
50,Aug 2 2022 3:53PM,243318,21,"NBTY Global, Inc."
51,Aug 2 2022 3:52PM,243317,21,"NBTY Global, Inc."
52,Aug 2 2022 3:50PM,243315,21,"NBTY Global, Inc."
53,Aug 2 2022 3:49PM,243314,15,"Carwin Pharmaceutical Associates, LLC"
54,Aug 2 2022 3:19PM,243309,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 3 2022 8:30AM,243310,10,"Methapharm, Inc.",,,19
1,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,,,9
2,Aug 3 2022 8:30AM,243316,10,"Nextsource Biotechnology, LLC",,,1
3,Aug 3 2022 8:30AM,243312,10,SugarBear,,,1
4,Aug 2 2022 4:04PM,243319,19,"AdvaGen Pharma, Ltd",,,20
5,Aug 2 2022 3:53PM,243318,21,"NBTY Global, Inc.",,,1
6,Aug 2 2022 3:52PM,243317,21,"NBTY Global, Inc.",,,1
7,Aug 2 2022 3:50PM,243315,21,"NBTY Global, Inc.",,,1
8,Aug 2 2022 3:49PM,243314,15,"Carwin Pharmaceutical Associates, LLC",,,1
9,Aug 2 2022 3:19PM,243309,10,Yusen – 3D Matrix,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 8:30AM,243310,10,"Methapharm, Inc.",19,,
1,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,9,,
2,Aug 3 2022 8:30AM,243316,10,"Nextsource Biotechnology, LLC",1,,
3,Aug 3 2022 8:30AM,243312,10,SugarBear,1,,
4,Aug 2 2022 4:04PM,243319,19,"AdvaGen Pharma, Ltd",20,,
5,Aug 2 2022 3:53PM,243318,21,"NBTY Global, Inc.",1,,
6,Aug 2 2022 3:52PM,243317,21,"NBTY Global, Inc.",1,,
7,Aug 2 2022 3:50PM,243315,21,"NBTY Global, Inc.",1,,
8,Aug 2 2022 3:49PM,243314,15,"Carwin Pharmaceutical Associates, LLC",1,,
9,Aug 2 2022 3:19PM,243309,10,Yusen – 3D Matrix,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 8:30AM,243310,10,"Methapharm, Inc.",19,,
1,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,9,,
2,Aug 3 2022 8:30AM,243316,10,"Nextsource Biotechnology, LLC",1,,
3,Aug 3 2022 8:30AM,243312,10,SugarBear,1,,
4,Aug 2 2022 4:04PM,243319,19,"AdvaGen Pharma, Ltd",20,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 8:30AM,243310,10,"Methapharm, Inc.",19.0,NaN,NaN
1,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,9.0,NaN,NaN
2,Aug 3 2022 8:30AM,243316,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
3,Aug 3 2022 8:30AM,243312,10,SugarBear,1.0,NaN,NaN
4,Aug 2 2022 4:04PM,243319,19,"AdvaGen Pharma, Ltd",20.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 8:30AM,243310,10,"Methapharm, Inc.",19.0,0.0,0.0
1,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,9.0,0.0,0.0
2,Aug 3 2022 8:30AM,243316,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
3,Aug 3 2022 8:30AM,243312,10,SugarBear,1.0,0.0,0.0
4,Aug 2 2022 4:04PM,243319,19,"AdvaGen Pharma, Ltd",20.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4622584,98.0,20.0,6.0
15,486488,3.0,21.0,14.0
19,486560,20.0,1.0,0.0
20,486420,4.0,12.0,10.0
21,1216427,3.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4622584,98.0,20.0,6.0
1,15,486488,3.0,21.0,14.0
2,19,486560,20.0,1.0,0.0
3,20,486420,4.0,12.0,10.0
4,21,1216427,3.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,98.0,20.0,6.0
1,15,3.0,21.0,14.0
2,19,20.0,1.0,0.0
3,20,4.0,12.0,10.0
4,21,3.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,98.0
1,15,Released,3.0
2,19,Released,20.0
3,20,Released,4.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,6.0,14.0,0.0,10.0,0.0
Executing,20.0,21.0,1.0,12.0,2.0
Released,98.0,3.0,20.0,4.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,6.0,14.0,0.0,10.0,0.0
1,Executing,20.0,21.0,1.0,12.0,2.0
2,Released,98.0,3.0,20.0,4.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,6.0,14.0,0.0,10.0,0.0
1,Executing,20.0,21.0,1.0,12.0,2.0
2,Released,98.0,3.0,20.0,4.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()